In [72]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import tensorflow as tf

In [111]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

X, y = make_classification(
    n_samples=10000, 
    n_features=20, 
    n_classes=2, 
    weights=[0.95, 0.05], #imbalanced dataset -  weights=[0.9, 0.1] creates 90% class 0 and only 10% class 1
    random_state=42
)

In [112]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data (Important for Neural Networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 1. with default bias value initialization (0)

In [113]:
model = Sequential()
model.add(Input(shape = (20,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # Binary classification

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [114]:
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1) 

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9080 - loss: 0.2688
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9490 - loss: 0.1404
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9553 - loss: 0.1263
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9565 - loss: 0.1208
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9572 - loss: 0.1176
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9582 - loss: 0.1152
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9588 - loss: 0.1132
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9590 - loss: 0.1113
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9595 - loss: 0.1097
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9591 - loss: 0.1082


In [115]:
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype("int32")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [116]:
print("--- Confusion Matrix ---")
print(confusion_matrix(y_test, y_pred))

print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred))

--- Confusion Matrix ---
[[1862   22]
 [  62   54]]

--- Classification Report ---
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1884
           1       0.71      0.47      0.56       116

    accuracy                           0.96      2000
   macro avg       0.84      0.73      0.77      2000
weighted avg       0.95      0.96      0.95      2000



# 2. changing the bias initialization o final layer to tackle imbalanced data 

In [117]:
neg, pos = np.bincount(y_train)
initial_bias = np.log([pos/neg])

print(f"Calculated Initial Bias: {initial_bias}")

model = Sequential()
model.add(Input(shape = (20,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))

# Apply the calculated bias to the output layer to handle imbalanced dataset

model.add(Dense(1, activation='sigmoid', bias_initializer=Constant(initial_bias)))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

Calculated Initial Bias: [-2.88300718]
Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9498 - loss: 0.1647
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9553 - loss: 0.1264
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9570 - loss: 0.1197
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9575 - loss: 0.1157
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9579 - loss: 0.1125
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9582 - loss: 0.1100
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9595 - loss: 0.1080
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9594 - loss: 0.1063
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9594 - loss: 0.1047
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9603 - loss: 0.1033


In [118]:
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype("int32")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [119]:
print("--- Confusion Matrix ---")
print(confusion_matrix(y_test, y_pred))

print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred))

--- Confusion Matrix ---
[[1870   14]
 [  71   45]]

--- Classification Report ---
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      1884
           1       0.76      0.39      0.51       116

    accuracy                           0.96      2000
   macro avg       0.86      0.69      0.75      2000
weighted avg       0.95      0.96      0.95      2000



In [ ]:
# here we can clearly see precision value is clearly incresed in class 1. it will increase more if run for more epochs.
# it will need more epochs than 